[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/liviucotfas/grey-systems-book/blob/main/grey-fixed-weight-clustering.ipynb)

In [2]:
import pandas as pd
df_objects = pd.read_excel("b2f-24rows-wf-1.xls")
df_white_weight_functions = pd.read_excel("b2f-24rows-wf-1.xls", "Sheet2")
df_ratios = pd.read_excel("b2f-24rows-wf-1.xls", "Sheet3")

In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import files
    uploaded = files.upload()


In [3]:
print(df_objects)
print(df_white_weight_functions)
print(df_ratios)

    Object\Parameter  boarding_s    aisle_s  windows_s   aisle_i    type3_i
0            Object1    852.4500  4659.3165  4424.0640  3.236732  11.856901
1            Object2    856.5975  4338.8280  4095.1140  3.561810  12.742409
2            Object3    862.5780  4033.5960  3811.8540  3.936406  14.038073
3            Object4    869.7105  3788.9250  3585.5865  4.251966  14.720846
4            Object5    872.9550  3568.1715  3334.9035  4.453164  15.292096
..               ...         ...        ...        ...       ...        ...
248        Object249    857.2500  3982.4445  3783.3960  4.084232  13.237604
249        Object250    847.4775  4023.8100  3815.6160  4.050924  13.180964
250        Object251    842.9565  4277.6685  4066.0020  3.882708  12.888646
251        Object252    841.2780  4306.4625  4065.3780  3.962878  12.977930
252        Object253    830.2185  4614.0720  4376.4240  3.665742  12.684596

[253 rows x 6 columns]
  Whitenization weight function\Parameter       boarding_s  \
0 

In [4]:
class WhitenizationWeightFunction(object):
    def __init__ (self, function_as_string : str):
        # split the string into turning points
        self.turning_points = function_as_string.split(",")
        # validate the number of turning points
        if len(self.turning_points) != 4:
            raise ValueError("The string should contain 4 turning ponits separated by `,`: x,x,x,x")
        # strip any white spaces 
        self.turning_points = list(map(lambda s:s.strip(), self.turning_points))
    def __call__(self, value : float) -> float:
        # 1. Whitenization weight function of lower measure
        if self.turning_points[0] == "-" and self.turning_points[1] == "-" and self.turning_points[2] != "-" and self.turning_points[3] != "-":
            return self.type1(value)
        # 2. Whitenization weight function of upper measure
        elif self.turning_points[0] != "-" and self.turning_points[2] == "-" and self.turning_points[3] == "-":
            return self.type2(value)
        # 3. Whitenization weight function of moderate measure
        elif self.turning_points[0] != "-" and self.turning_points[1] != "-" and self.turning_points[2] == "-" and self.turning_points[3] != "-":
            return self.type3(value)
        # 4. Typical whitenization weight function
        elif self.turning_points[0] != "-" and self.turning_points[1] != "-" and self.turning_points[2] != "-" and self.turning_points[3] != "-":
            return self.type4(value)
        else:
            raise ValueError()            
    def type1(self, value):
        if value < 0.0 or value > float(self.turning_points[3]):
            return 0.0
        elif value < float(self.turning_points[2]) and value >= 0.0:
            return 1.0
        elif value >= float(self.turning_points[2]) and value < float(self.turning_points[3]):
            return (float(self.turning_points[3]) - value) / (float(self.turning_points[3]) - float(self.turning_points[2]));
    def type2(self, value):
        if value < float(self.turning_points[0]):
            return 0.0
        elif value >= float(self.turning_points[0]) and value < float(self.turning_points[1]):
            return (value - float(self.turning_points[0])) / (float(self.turning_points[1]) - float(self.turning_points[0]));
        elif value >= float(self.turning_points[1]):
            return 1.0
    def type3(self, value):
        if value < float(self.turning_points[0]) or value > float(self.turning_points[3]):
            return 0
        elif value >= float(self.turning_points[0]) and value < float(self.turning_points[1]):
            return (value - float(self.turning_points[0])) / (float(self.turning_points[1]) - float(self.turning_points[0]))
        elif value >= float(self.turning_points[1]) and value <= float(self.turning_points[3]):
            return (float(self.turning_points[3]) - value) / (float(self.turning_points[3]) - float(self.turning_points[1]))
    def type4(self, value):
        if value < float(self.turning_points[0]) or value > float(self.turning_points[3]):
            return 0
        elif value >= float(self.turning_points[0]) and value <= float(self.turning_points[1]):
            return (value - float(self.turning_points[0])) / (float(self.turning_points[1]) - float(self.turning_points[0]));
        elif value >= float(self.turning_points[1]) and value < float(self.turning_points[2]):
            return 1
        elif value >= float(self.turning_points[2]) and value <= float(self.turning_points[3]):
            return (float(self.turning_points[3]) - value) / (float(self.turning_points[3]) - float(self.turning_points[2]));

text = "-,-,841,859"
f1 =  WhitenizationWeightFunction(text)
f1(5)

        



1.0

In [5]:
# Starting from the df_white_weight_functions create a list of lists holding WhitenizationWeightFunction 
whitenization_weight_functions = []
for i in range(df_white_weight_functions.shape[0]):
    whitenization_weight_functions.append([])
    for j in range(df_white_weight_functions.shape[1] - 1):
        function_as_string = df_white_weight_functions.iloc[i, j+1]
        whitenization_weight_functions[i].append(WhitenizationWeightFunction(function_as_string))

In [6]:
no_of_obj = len(df_objects)
no_of_obj #length1

253

In [7]:
no_of_param = len(df_objects.columns) - 1 # we substract 1 becuase the first column includes the names
no_of_param #length2

5

In [8]:
no_of_functions = len(df_white_weight_functions)
no_of_functions #length3

3

In [9]:
# Compute the clustering coefficents

# Start by creating a list of lists filled with 0
clustering_coefficents = []
for i in range(no_of_obj):
    clustering_coefficents.append([])
    for j in range(no_of_functions): 
        clustering_coefficents[i].append(0)

# Acctually compute the values
for i in range(no_of_functions):
  for j in range(no_of_obj):
    result = 0

    for k in range(no_of_param):
      value = df_objects.iloc[j, k+1] # +1 becuase the first column includes a description
      whitenization_weight_function = whitenization_weight_functions[i][k]

      function_value = whitenization_weight_function(value)
      ratio_value = df_ratios.iloc[0, k+1] # +1 becuase the first column includes a description

      result +=  function_value* ratio_value
      
    clustering_coefficents[j][i] = result; 

In [10]:
clustering_coefficents

[[0.3638888888888927, 0.6361111111111073, 0.0],
 [0.133472222222224, 0.866527777777776, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1, 0.0],
 [0.0, 1, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 0.9523055555555529, 0.047694444444447176],
 [0.0, 0.9807222222222296, 0.01927777777777035],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.36163888888888907, 0.6383611111111109, 0.0],
 [0.8077222222222278, 0.19227777777777216, 0.0],
 [1.0, 0.0, 0.0],
 [1.0, 0, 0.0],
 [1.0, 0, 0.0],
 [0.06297222222222369, 0.9370277777777763, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 1.0, 0.0],
 [0.0, 0.555805555555556, 0.44419444444444406],
 [0.0, 0.4398888888888892, 0.5601111111111108],
 [0.0, 0.458638888888888, 0.5413611111111121],
 [0.0, 0.4960555555555541, 0.5039444444444459],
 [0.0, 0.3052222222222238, 0.6947777777777762],
 [0.0, 0.37130555555556083, 0.6286944444444392],
 [0.0, 0.478472222222226, 0.521527777777774],
 [0.0, 0.6

In [11]:
cluster = []

for coefficents in clustering_coefficents:
    max = coefficents[0]
    index = 0
    for i in range(1,len(coefficents)):
        if coefficents[i] > max:
            max = coefficents[i]
            index = i
    cluster.append(index)

In [12]:
df_results = df_objects.copy()
df_results["cluster"] = cluster

In [13]:
df_results

,Object\Parameter,boarding_s,aisle_s,windows_s,aisle_i,type3_i,cluster
0,Object1,852.4500,4659.3165,4424.0640,3.236732,11.856901,1
1,Object2,856.5975,4338.8280,4095.1140,3.561810,12.742409,1
2,Object3,862.5780,4033.5960,3811.8540,3.936406,14.038073,1
3,Object4,869.7105,3788.9250,3585.5865,4.251966,14.720846,1
4,Object5,872.9550,3568.1715,3334.9035,4.453164,15.292096,1
...,...,...,...,...,...,...,...
248,Object249,857.2500,3982.4445,3783.3960,4.084232,13.237604,1
249,Object250,847.4775,4023.8100,3815.6160,4.050924,13.180964,0
250,Object251,842.9565,4277.6685,4066.0020,3.882708,12.888646,0
251,Object252,841.2780,4306.4625,4065.3780,3.962878,12.977930,0


In [16]:
for i in range(no_of_functions):
    cluster_index = i+1
    object_indexs = str(list(df_results[df_results['cluster'] == i].index + 1)) # +1 to have the indexes starting from 1
    print(str(cluster_index) + ": "+object_indexs)

1: [19, 20, 21, 22, 40, 41, 42, 43, 61, 62, 63, 81, 82, 99, 250, 251, 252, 253]
2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 23, 24, 25, 26, 33, 34, 35, 36, 37, 38, 39, 44, 45, 46, 55, 56, 57, 58, 59, 60, 64, 65, 76, 77, 78, 79, 80, 83, 96, 97, 98, 100, 101, 114, 115, 116, 117, 118, 130, 131, 132, 133, 146, 147, 148, 160, 161, 162, 173, 174, 175, 186, 187, 197, 198, 199, 207, 208, 209, 216, 217, 218, 224, 225, 226, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249]
3: [27, 28, 29, 30, 31, 32, 47, 48, 49, 50, 51, 52, 53, 54, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 176, 177, 178, 179, 180, 181, 1